scanpy installation https://scanpy.readthedocs.io/en/stable/installation.html

In [ ]:
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc

In [ ]:
main_path = '/Users/example'

# Import CSV data

In [ ]:
filename = 'filename.csv'
csv_filepath = os.path.join(main_path, filename)
csv_file = pd.read_csv(csv_filepath) 

# Convert CSV to anndata

reference: https://falexwolf.de/blog/171223_AnnData_indexing_views_HDF5-backing/

Assumption here is that protein markers are column names, cells are rows.
Also, any sample information like donor and treatment are stored in additional columns in that same file.

"variable_names" list contains column names that are marker names

In [ ]:
variable_names= ['CD45', 'CD19', 'CD16']

In [ ]:
var = pd.DataFrame(index=variable_names)

store median intensity values in dataframe "X"

In [ ]:
X = csv_file.loc[:, variable_names]

generate annotated data

In [ ]:
adata = ad.AnnData(X, var=var)

Store additional file information under anndata observations "obs".
If cells have already been clustered, cluster ids should be stores under observations.

In [ ]:
adata.obs_names_make_unique()
adata.obs['donor'] = csv_file['donor'].values
adata.obs['treatment'] = csv_file['treatment'].values

save anndata file

In [ ]:
adata_path = os.path.join(main_path, 'adata_filename.h5ad')
adata.write(adata_path)

# Import anndata and analyze

In [ ]:
adata_path = os.path.join(main_path, 'adata_filename.h5ad')
adata = sc.read_h5ad(adata_path)

Optional: raw median intensity data can be stored under "adata.raw" and median intensities to use in downstream analysis can be stored under "adata.X"

In [ ]:
adata.raw = adata
adata.X = np.arcsinh(adata.X/5)
adata.write(adata_path) #save

or data can be normalized

In [ ]:
sc.pp.scale(adata, zero_center=True, max_value=None)

determine protein markers that will be used to generate clusters

In [ ]:
clustering = ['B220','MHCII', 'CD3', 'CD16']

In [ ]:
adata.obsm['cluster_channels']= adata[:, clustering].X

determine size of local neighborhood

In [ ]:
sc.pp.neighbors(adata, n_neighbors = 5, use_rep= 'cluster_channels', random_state = 1)

run umap

In [ ]:
sc.tl.umap(adata, random_state = 1)

run leiden clustering (choose resolution, 1 = more granular clusters)

In [ ]:
sc.tl.leiden(adata, resolution=1, key_added = 'leiden1', random_state = 1)

visualize umap with superimposed clusters

In [ ]:
sc.pl.umap(adata, color = 'leiden1')

Run PAGA based on current neighborhoods, conduct analysis on current leiden clusters. Alternatively, if already clustered, run neighborhood analysis again on protein markers of interest.

In [ ]:
sc.tl.paga(adata, groups='leiden1')

plot PAGA, determine threshold of connectivities between clusters

In [ ]:
sc.pl.paga(adata, color=['leiden1'], threshold=0.4)

alternatively more detailed plot

In [ ]:
sc.tl.draw_graph(adata, init_pos='paga')

In [ ]:
sc.pl.paga_compare(
    adata, threshold=0.4, title='', right_margin=0.2, size=10, edge_width_scale=0.5,
    legend_fontsize=12, fontsize=12, frameon=False, edges=True)

In [ ]:
adata.write(adata_path) #save